# OMSF 2025: Joint Demo



Lets imagine we are we are computational medicinal chemists assisting a project team working on ligand design for a biological target, in this case **MCL1** a protein that is a well established target for oncology. 

In [ ]:
import nglview

nglview.show_file("source/openfe/protein.pdb")

In [ ]:
import pandas

design_set = pandas.read_csv("source/openadmet/ligands.csv")
design_set

In [ ]:
import datamol

design_set["mol"] = design_set["SMILES"].apply(datamol.to_mol)
datamol.to_image(design_set["mol"])

In [ ]:
!openadmet predict \
    --input-path ./source/openadmet/ligands.csv \
    --input-col SMILES \
    --model-dir ./source/openadmet/cyp3a4_anvil_lgbm \
    --model-dir ./source/openadmet/cyp2d6_anvil_lgbm \
    --model-dir ./source/openadmet/cyp2c9_anvil_lgbm \
    --model-dir ./source/openadmet/cyp1a2_anvil_lgbm \
    --output-path ./predictions.csv

In [ ]:
predictions = pandas.read_csv("predictions.csv")
predictions.sort_values("OADMET_PRED_openadmet-CYP3A4-pchembl-lgbm", ascending=False)

In [ ]:
CYP3A4_THRESHOLD = 5.6  # is this actually <=> IC50 of 5 micromolar?

mask = predictions["OADMET_PRED_openadmet-CYP3A4-pchembl-lgbm"] < CYP3A4_THRESHOLD

In [ ]:
keep = predictions[mask]
keep

In [ ]:
!openfe plan-rbfe-network \
    --protein source/openfe/protein.pdb \
    --molecules source/openfe/ligands_charged.sdf \
    --settings source/openfe/settings.yaml \
    --output-dir rbfe/

In [ ]:
!tree rbfe/

In [ ]:
!#openfe quickrun rbfe/transformations/rbfe_ligand_14_complex_ligand_13_complex.json -o results_complex.json -d working-directory/

In [ ]:
!openfe gather source/openfe/results_jsons/

### Comparing predictions of different CYP models

In [ ]:
import seaborn
from matplotlib import pyplot


seaborn.kdeplot(data=predictions, fill=True)
pyplot.xlabel("predicted pChEMBL")
pyplot.title("Distribution of predicted pChEMBL values for CYP antitargets")

pyplot.show()

In [ ]:
CYP1A2_pChEMBL_data = pandas.read_csv("./source/openadmet/cyp1a2_chembl_permissive.csv")
CYP1A2_pChEMBL_data["target"] = "CYP1A2"
CYP3A4_pChEMBL_data = pandas.read_csv("./source/openadmet/cyp3a4_chembl_permissive.csv")
CYP3A4_pChEMBL_data["target"] = "CYP3A4"
CYP2C9_pChEMBL_data = pandas.read_csv("./source/openadmet/cyp2c9_chembl_permissive.csv")
CYP2C9_pChEMBL_data["target"] = "CYP2C9"
CYP2D6_pChEMBL_data = pandas.read_csv("./source/openadmet/cyp2d6_chembl_permissive.csv")
CYP2D6_pChEMBL_data["target"] = "CYP2D6"

combined = pandas.concat(
    [
        CYP2D6_pChEMBL_data,
        CYP2C9_pChEMBL_data,
        CYP1A2_pChEMBL_data,
        CYP3A4_pChEMBL_data,
    ]
)

In [ ]:
seaborn.kdeplot(combined, x="pchembl_value_mean", hue="target")

pyplot.title("Distribution of CYP pChEMBL values (ChEMBL 35)")
pyplot.xlabel("Target pChEMBL")

pyplot.show()

### More about the OpenFE CLI

In [ ]:
# https://docs.openfree.energy/en/stable/reference/cli/charge_molecules.html
!openfe charge-molecules \
    --molecules source/openfe/ligands.sdf \
    --output out.sdf \
    --n-cores 12

In [ ]:
%%html
<img src="source/openfe/network_layouts.png">

In [ ]:
!cat source/openfe/settings.yaml

In [ ]:
from konnektor.visualization import draw_ligand_network
import openfe

In [ ]:
draw_ligand_network(
    openfe.setup.LigandNetwork.from_graphml(open('rbfe/ligand_network.graphml').read()),
    node_size=3000);

In [ ]:
!cat source/openfe/radial.yaml

In [ ]:
!openfe plan-rbfe-network \
    --protein source/openfe/protein.pdb \
    --molecules source/openfe/ligands_charged.sdf \
    --settings source/openfe/radial.yaml \
    --output-dir radial

In [ ]:
draw_ligand_network(
    openfe.setup.LigandNetwork.from_graphml(open('radial/ligand_network.graphml').read()),
    node_size=3000);

## Cool stuff you can do with OpenFF

### RDKit to MD simulation in seconds

In [ ]:
from rdkit import Chem

rdmol = Chem.MolFromMolFile("source/openff/aspirin.sdf")
print(Chem.MolToSmiles(rdmol))
Chem.Draw.MolToImage(rdmol)

In [ ]:
from openff.toolkit import ForceField, Molecule

def simulate_and_visualize(
    rdmol: Chem.Mol,
    force_field: ForceField,
) -> "nglview.NGLWidget":
    import mdtraj
    import openmm
    import openmm.unit

    interchange = force_field.create_interchange(
        Molecule.from_rdkit(rdmol).to_topology(),
    )

    integrator = openmm.LangevinMiddleIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        0.002 * openmm.unit.picoseconds,
    )

    simulation = interchange.to_openmm_simulation(integrator)

    # OpenMM setup boilerplate
    simulation.minimizeEnergy(tolerance=100)
    dcd_reporter = openmm.app.DCDReporter(file="trajectory.dcd", reportInterval=10)
    simulation.reporters.append(dcd_reporter)
    simulation.step(5000)

    # these lines would be good to shove into a utils.py file
    # ... def viz(trj_file: str, top_something) -> nglview.NGLWidget
    # Visualize the trajectory
    trajectory: mdtraj.Trajectory = mdtraj.load(
        "trajectory.dcd",
        top=mdtraj.Topology.from_openmm(interchange.topology.to_openmm()),
    )
    view = nglview.show_mdtraj(trajectory)
    view.add_representation("line", selection="protein")
    view.add_line(selection="water")

    return view

sage = ForceField("openff-2.2.1.offxml")

simulate_and_visualize(rdmol, sage)

### Run protein-ligand complexes with Sage + ff14SB

In [ ]:
from openff.toolkit import Topology

topology = Topology.from_pdb(
    "source/openff/complex_topology.pdb",
    unique_molecules=[Molecule.from_smiles("c12c(Cl)cccc1sc(C(=O)[O-])c(Cl)2")],
)

protein = topology.molecule(0)
ligand = topology.molecule(1)

topology.visualize()

In [ ]:
from pdbfixer import PDBFixer
import openmm.unit
import pathlib

if not pathlib.Path("receptor_solvated.pdb").exists():
    topology.to_file('temp.pdb')
    fixer = PDBFixer('temp.pdb')
    fixer.addSolvent(
        padding=0.5 * openmm.unit.nanometer,
        ionicStrength=0.15 * openmm.unit.molar
    )

    with open("receptor_solvated.pdb", "w") as f:
        openmm.app.PDBFile.writeFile(fixer.topology, fixer.positions, f)

solvated_topology = Topology.from_pdb(
    "receptor_solvated.pdb", 
    unique_molecules=[ligand],
)

solvated_topology.visualize()


In [ ]:
from openff.interchange import Interchange


if not pathlib.Path("interchange.json").exists():
    sage_with_ff14sb= ForceField("openff-2.2.1.offxml", "ff14sb_off_impropers_0.0.4.offxml")

    interchange = sage_with_ff14sb.create_interchange(solvated_topology)

    with open('interchange.json', 'w') as f:
        f.write(interchange.json())

interchange = Interchange.model_validate_json(open("interchange.json").read())
interchange.visualize()

In [ ]:
if not pathlib.Path("protein-ligand.dcd").exists():
    integrator = openmm.LangevinMiddleIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        0.002 * openmm.unit.picoseconds,
    )

    simulation = interchange.to_openmm_simulation(integrator)

    simulation.minimizeEnergy(tolerance=100)

    dcd_reporter = openmm.app.DCDReporter(
        file="protein-ligand.dcd", 
        reportInterval=100,
    )
    simulation.reporters.append(dcd_reporter)

    # ~1 minute runtime on a modern laptop
    simulation.step(5000)

In [ ]:
import mdtraj


nglview.show_mdtraj(
    mdtraj.load(
        "protein-ligand.dcd",
        top=mdtraj.Topology.from_openmm(solvated_topology.to_openmm()),
    )
)

### Simulating post-translational modifications

In [ ]:
from ipywidgets import Image
from openff.toolkit import ForceField, Molecule
from rdkit.Chem import Draw
from rdkit.Chem.rdChemReactions import ReactionFromSmarts
from openff.pablo import CCD_RESIDUE_DEFINITION_CACHE

cysteine = CCD_RESIDUE_DEFINITION_CACHE["CYS"][0].to_openff_molecule()


In [ ]:
thiol_maleimide_click_smarts = (
    "[C:10]-[S:1]-[H:2]"
    + "."
    + "[N:3]1-[C:4](=[O:5])-[C:6](-[H:11])=[C:7](-[H:12])-[C:8](=[O:9])-1"
    + ">>"
    + "[N:3]1-[C:4](=[O:5])-[C:6](-[H:2])(-[H:11])-[C@:7](-[S:1]-[C:10])(-[H:12])-[C:8](=[O:9])-1"
)

rxn = ReactionFromSmarts(thiol_maleimide_click_smarts)
d2d = Draw.MolDraw2DCairo(800, 300)
d2d.DrawReaction(
    ReactionFromSmarts(thiol_maleimide_click_smarts), highlightByReactant=True
)
Image(value=d2d.GetDrawingText())


In [ ]:
from ptm_prototype import react
from openff.pablo import ResidueDefinition
from openff.pablo.chem import PEPTIDE_BOND
from openff.pablo import CCD_RESIDUE_DEFINITION_CACHE, topology_from_pdb

maleimide = Molecule.from_file("source/openff/maleimide.sdf")
maleimide.generate_unique_atom_names()

products = list(react([cysteine, maleimide], thiol_maleimide_click_smarts))
dye = products[0][0]
name_corrections = {
    4:  "H3x", 25: "C9x", 26: "H4x", 28: "C8x", 29: "H6x", 30: "H5x",
    32: "C10x", 33: "C11x", 34: "O2x", 35: "O3x", 36: "C23x", 37: "C12x",
    38: "C22x", 39: "C18x", 40: "C13x", 41: "C17x", 42: "H14x", 43: "C21x",
    44: "C19x", 45: "O5x", 46: "C14x", 47: "H7x", 49: "H13x", 50: "C20x",
    51: "H11x", 52: "C15x", 53: "H8x", 54: "H10x", 55: "O6x",
}
for i, name in name_corrections.items():
    dye.atom(i).name = name

dye_resdef = ResidueDefinition.from_molecule(
    molecule=dye,
    residue_name="DYE",
    linking_bond=PEPTIDE_BOND,
)

topology = topology_from_pdb(
    "source/openff/3ip9_dye_solv.pdb",
    residue_database=CCD_RESIDUE_DEFINITION_CACHE.with_({"DYE": [dye_resdef]}),
)
w = topology.visualize()
w.clear_representations()
w.add_cartoon()
w.add_line(opacity=0.5, crossSize=1.0)
w.add_licorice("DYE", radius=0.3)
w.add_unitcell()
w.center("DYE")
w


In [ ]:
from ptm_prototype import parametrize_with_nagl, simulate_and_visualize

import pathlib

if not pathlib.Path("ptm.dcd").exists():
    sage_ff14sb = ForceField("openff-2.2.1.offxml", "ff14sb_off_impropers_0.0.4.offxml")
    interchange = parametrize_with_nagl(force_field=sage_ff14sb, topology=topology)

    simulate_and_visualize(interchange)

In [ ]:
import mdtraj
import nglview

trajectory: mdtraj.Trajectory = mdtraj.load(
    "ptm.dcd",
    top=mdtraj.Topology.from_openmm(topology.to_openmm())
)

view = nglview.show_mdtraj(trajectory)

view.clear_representations()
view.add_cartoon()
view.add_line(opacity=0.5, crossSize=1.0)
view.add_licorice("DYE", radius=0.3)
view.add_unitcell()
view.center("DYE")
view

### Rapidly charge large ligands with NAGL

In [ ]:
from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper
from openff.toolkit.utils.ambertools_wrapper import AmberToolsToolkitWrapper
from openff.toolkit import Molecule


ambertools_wrapper = AmberToolsToolkitWrapper()

nagl_wrapper = NAGLToolkitWrapper()
nagl_wrapper.assign_partial_charges(Molecule.from_smiles('C'), "openff-gnn-am1bcc-0.1.0-rc.3.pt")

# TODO: Curate a dataset with larger range (>100 heavy atoms)
# TODO: Only run AmberTools on molecules smaller than a certain number of heavy atoms
ligands = [Molecule.from_smiles(smiles) for smiles in pandas.read_csv("source/openadmet/ligands.csv")['SMILES']]

In [27]:
import time
import pathlib

def charge_with_nagl(molecule) -> float:
    start = time.time()
    nagl_wrapper.assign_partial_charges(molecule, "openff-gnn-am1bcc-0.1.0-rc.3.pt")
    end = time.time()
    return end - start

def charge_with_ambertools(molecule) -> float:
    start = time.time()
    ambertools_wrapper.assign_partial_charges(molecule, "am1bcc")
    end = time.time()
    return end - start

In [28]:
import pandas


if not pathlib.Path("nagl_timings.csv").exists():
    nagl_timings = pandas.DataFrame(columns=["n_atoms", "SMILES", "time"])

    for molecule in ligands:
        nagl_timings.loc[len(nagl_timings)] = (molecule.n_atoms, molecule.to_smiles(), charge_with_nagl(molecule))

    #for molecule in Molecule.from_file("source/openff/MiniDrugBank.sdf")[:10]:
    #    nagl_timings.loc[len(nagl_timings)] = (molecule.n_atoms, charge_with_nagl(molecule))

if not pathlib.Path("ambertools_timings.csv").exists():
    ambertools_timings = pandas.DataFrame(columns=["n_atoms", "SMILES", "time"])

    for index, molecule in enumerate(ligands):
        ambertools_timings.loc[len(ambertools_timings)] = (molecule.n_atoms, molecule.to_smiles(), charge_with_ambertools(molecule))

In [ ]:
import seaborn
from matplotlib import pyplot

seaborn.regplot(nagl_timings, x="n_atoms", y="time", label="NAGL", scatter=True)
seaborn.regplot(ambertools_timings, x="n_atoms", y="time", order=3, label="AmberTools", scatter=True)

pyplot.ylim((0, 40))
pyplot.legend(loc=0)


## Credits